## Web Scrapping (Selinium)

In [144]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

Q1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [145]:
# connect to webdriver
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)

# to get url
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : guitars


In [147]:
# to Locate searc_bar by id
search_bar = driver.find_element_by_id("twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()              

Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [148]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the laptop
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [149]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Ratings = []
No_Ratings = []
Price = []
Return = []
Expected_Delivery = []
Availability = []
Other_Details = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping product name
    try:
        prod=driver.find_element_by_xpath("//span[@id='productTitle']")
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping brand name
    try:
        brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
     #Scraping Ratings
    try:
        rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
        Ratings.append(rat.get_attribute("title"))   
    except NoSuchElementException as e:
        Ratings.append("-")
        
        
    #Scraping No of Ratings
    try:
        no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']/span")
        No_Ratings.append(no_rat.text)
    except NoSuchElementException as e:
        No_Ratings.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element_by_xpath('//td[@class="a-span12"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element_by_xpath("//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element_by_xpath("//div[@id='availability']/span")
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
        
        
    #Scraping Other_Details
    try:
        details=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        Other_Details.append(details.text)
    except NoSuchElementException as e:
        Other_Details.append("-")
        
        
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Rating'] = Ratings
Guitar['No of Ratings'] = No_Ratings
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Other_Details'] = Other_Details
Guitar['Urls'] = urls

In [150]:
#Print dataframe
Guitar

,Name,Brand,Rating,No of Ratings,Price,Return,Expected_Delivery,Availability,Other_Details,Urls
0,Kadence Frontier guitar with Online Guitar lea...,Visit the Kadence Store,4.0 out of 5 stars,"1,481 ratings","₹5,399.00",7 Days Replacement,"Thursday, May 26",,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,Visit the Kadence Store,4.0 out of 5 stars,"1,215 ratings","₹7,348.00",7 Days Replacement,-,In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",Visit the blueberry Store,3.8 out of 5 stars,14 ratings,-,7 Days Replacement,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/gp/slredirect/picassoRed...
3,Vault Traveller 34 Inch Acoustic Guitar With B...,Visit the VAULT Store,3.9 out of 5 stars,342 ratings,-,7 Days Replacement,-,In stock.,"Size: 34"" Steel stringed | Number of Strings: ...",https://www.amazon.in/gp/slredirect/picassoRed...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,4.0 out of 5 stars,"15,871 ratings",-,7 Days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
171,Revel 38 Inches Cutaway Design Acoustic Guitar...,Brand: Revel,5.0 out of 5 stars,1 rating,-,7 Days Replacement,-,In stock.,Great looks with an innovative design for easy...,https://www.amazon.in/Revel-RVL-38C-LGP-NT-Lin...
172,"KHUSHI MUSICALS Finger Sleeve, 4/8 PCS Silicon...",Brand: KHUSHI MUSICALS,-,-,-,7 Days Replacement,-,In stock.,Silicone finger thumb picks covers: comfortabl...,https://www.amazon.in/KHUSHI-MUSICALS-Finger-S...
173,Ketostics® Givson Venus Super Special semi ele...,Brand: KETOSTICS,4.0 out of 5 stars,47 ratings,-,7 Days Replacement,-,In stock.,Venus Special 6 Strings | Acoustic Guitar | Co...,https://www.amazon.in/Ketostics%C2%AE-Special-...
174,Blueberry B-D38 Sunburst 38 Inch Acoustic Guit...,Visit the blueberry Store,3.5 out of 5 stars,107 ratings,-,7 Days Replacement,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/Blueberry-B-D38-Sunburst...


In [156]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="f331d6a2340dffce0c23bdd76858b7d9")>>

Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [154]:
# connect to web browser
driver = webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe")
# getting url
url = "https://images.google.com/"
#Creating empty list 
urls = []    
data = []
# creating loop for all the items to be serached
search_item = ["fruits", "cars", "Machine Learning","Guitar","Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    search_bar = driver.find_element_by_tag_name("input") #Xpath for search bar
    
    search_bar.send_keys(str(item)) #sending key word for search item
    
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,100)")
     
    # to get image url
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])
    
    # To save downloaded pics in local folder
    file = open(r"C:\Users\user\Desktop\fliprobo\google pics"+str(i)+".jpg", "wb")

    file.write(response.content)

In [155]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="f331d6a2340dffce0c23bdd76858b7d9")>>

Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [178]:
# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe")

# To get the web page 
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

In [179]:
# to close login pop up
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()

In [180]:
# to enter the item name which user wants to search
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the name of Smartphone that has to be searched : Oneplus Nord


In [181]:
# To get  urls of phones on 1st page
page1_urls = []
urls = driver.find_elements_by_class_name('s1Q9rs')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [182]:
len(page1_urls)

40

In [183]:
# to make emoty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Display Resolution"] = []
Smartphones["Processor"] = []
Smartphones["Processor Cores"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [184]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
    
    #Scraping display resolution data
    try:
        dires_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if dires_tags.text != "Display Features" : raise NoSuchElementException
        disp_reso = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')    
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
    
    
    
    #Scraping Processor data
    try:
        pro_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_tags.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
        
    
    
    #Scraping Processor core data    
    try:                                                                                     
        core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_tags.text != "Processor Core" :
            core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_tags.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        Smartphones['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
    
    
    
    #Scraping battery capacity data
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')
        

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGDWF8GJTFAXVH&lid=LSTMOBGDWF8GJTFAXVHN22GLT&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=1a0a319c-24e0-4fae-a8a4-1f5ae6d1bf16.MOBGDWF8GJTFAXVH.SEARCH&ppt=hp&ppn=homepage&ssid=1wmerejum80000001653233102593&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-2-5g-gray-sierra-128-gb/p/itmc88e4a702db46?pid=MOBGDBFFA5EGECHQ&lid=LSTMOBGDBFFA5EGECHQQKVBBR&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=1a0a319c-24e0-4fae-a8a4-1f5ae6d1bf16.MOBGDBFFA5EGECHQ.SEARCH&ppt=hp&ppn=homepage&ssid=1wmerejum80000001653233102593&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-2-5g-blue-haze-128-gb/p/itmc88e4a702db46?pid=MOBGDBFJXFJPVWKA&lid=LSTMOBGDBFJXFJPVWKAHB629I&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_3&otracker=sea

In [185]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))

40 40 40 40 40


In [186]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),2412 x 1080$$pixel,-,Octa Core,-,"₹19,942",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord 2 5G,Gray Sierra,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),1080 x 2400$$pixels,-,Dual Core,-,"₹29,200",https://www.flipkart.com/oneplus-nord-2-5g-gra...
2,OnePlus,Nord 2 5G,Blue Haze,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),1080 x 2400$$pixels,-,Dual Core,-,"₹29,564",https://www.flipkart.com/oneplus-nord-2-5g-blu...
3,OnePlus,Nord 2 5G,PAC-MAN,12 GB,256 GB,Primary Camera,-,16.33 cm (6.43 inch),2400 x 1080$$pixel,-,Octa Core,-,"₹37,890",https://www.flipkart.com/oneplus-nord-2-5g-pac...
4,Micvir,OnePlus Nord 2 5G,Black,-,-,-,-,-,-,-,-,-,₹179,https://www.flipkart.com/micvir-back-cover-one...
5,NewSelect,OnePlus Nord,Transparent,-,-,-,-,-,-,-,-,-,₹179,https://www.flipkart.com/newselect-back-cover-...
6,KWINE,"OnePlus Nord CE 2 5G, One Plus Nord CE 2, One ...",Black,-,-,-,-,-,-,-,-,-,₹249,https://www.flipkart.com/kwine-case-back-cover...
7,Mobile,OnePlus Nord CE 2 5G,Black,-,-,-,-,-,-,-,-,-,₹229,https://www.flipkart.com/mobile-mart-back-cove...
8,GoldKart,OnePlus Nord CE 2 5G,Black,-,-,-,-,-,-,-,-,-,₹229,https://www.flipkart.com/goldkart-back-cover-o...
9,COSHIBA,Front,ONEPLUS NORD,-,-,-,-,-,-,-,-,-,₹175,https://www.flipkart.com/coshiba-edge-tempered...


In [187]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="eca134cfc79ac5a02adae70b506b2e09")>>

Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [142]:
# to get web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)

# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id("searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element_by_id("searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Gujrat
URL Extracted:  https://www.google.co.in/maps/place/Gujarat/@22.3545119,66.8184489,6z/data=!3m1!4b1!4m5!3m4!1s0x3959051f5f0ef795:0x861bd887ed54522e!8m2!3d22.258652!4d71.1923805
Latitude = 22.3545119, Longitude = 66.8184489


In [188]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="eca134cfc79ac5a02adae70b506b2e09")>>

Q6.Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [189]:
# to get web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)

#to get url
driver.get('https://trak.in/')

#to get xpath for funding deals
fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [190]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [191]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="c34010c4f033c946d8c16e0287c7a0cc")>>

Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [192]:
# to getb web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)

#to get url
url = "https://www.digit.in/"
driver.get(url)
time.sleep(3)
#to search for best laptop
best_gam_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [193]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []
# to Scrape laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
# to Scrape the operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


# to Scrape display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#to Scrape Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


# to Scrape memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


# to Scrape weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#to Scrape dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


# to Scrape Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#to Scrape price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [194]:
# to make dataframe
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [195]:
#Printing data frame
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,MSI Raider GE76,Windows 11 Home,"17"" (3840 x 2160)",12th Gen Intel Core i9-12900HK | 5 GHz,2 TB SSD/16 GBGB DDR5,2.9,397 x 284 x 26,NVIDIA GeForce RTX 3080Ti,"Rs 429,940"
1,ASUS ROG Strix Scar 15,Windows 11 Home,"15.6"" (2560 x 1440)",12th Gen Intel Core i9-12900H | 2.5 GHz,2 TB SSD/32 GBGB DDR5,2.3,259 x 354 x 27,NVIDIA GeForce RTX 3070 Ti,"Rs 285,390"
2,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
3,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
4,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 193,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 144,990"
7,HP Omen 16,Windows 11 Home,"16.1"" (1920 x 1080)",12th Gen Intel Core i7-12700H | 4.7 GHz,1 TB SSD/16 GBGB DDR4,2.32,36.92 x 24.8 x 2.3,NVIDIA GeForce RTX 3060,"Rs 139,990"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 53,490"


In [196]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="2f1e99a8baeefe39c4832d4835e101d6")>>

Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [197]:
# To get web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)

# to get url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(2)

#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billioners 
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()
time.sleep(1)

#select world billioners 
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()
time.sleep(1)

In [198]:
# empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #to Scrape rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #to Scrape names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #to Scrape age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #to Scrape citizenship
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #to Scrape source of income
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #to Scrape Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #to scrape net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #to Click on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#to Scrape net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [199]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry

In [200]:
#Printing dataframe
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [201]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="dac3699e04e0180f8269fdc54d02bbb9")>>

Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [202]:
# to get web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(2)
# to get url
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [203]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element_by_xpath('//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : RRR


In [204]:
#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [205]:
#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

In [206]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements_by_id("content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

['ఆడియో సాంగ్ రిలీజ్ అయినప్పుడు ఏం రెస్పాన్స్ రాలేదు..కానీ సినిమాలో ఎన్టీఆర్ నటన చూసిన తర్వాత ది బెస్ట్ సాంగ్ అయిపోయింది  jai NTR 🔥 🌊',
 'ఆయుధాలే కాదు..... పాట కూడా ప్రజలతో యుద్ధం చేయించగలదు అని నిరూపించిన RRR team కి మరియు మా గోండు బెబ్బులి మా NTR అన్న కి శతకోటి దండాలు 👏👏👏👏👏',
 'Its NTR revolution  ప్రపంచ సిని చరిత్రలో  నటనె ఊపిరి గా  జీవిస్తున్న  గొప్ప వ్యక్తి  మా అన్న తారకరాముడు 🙏🙏🙏😥😥😥',
 'NTR fans అయినందుకు చాలా గర్వంగా ఉంది\nజై NTR Anna 🌊🌊🌊🌊🌊💐💐💐💐🙏🙏🙏💐🙏🙏🙏🙏🙏🙏',
 'ఈ పాట విన్నంత సేపు కళ్ళలో నీళ్ళు ఆగలేదు 😭😭😭\nఈ పాట రాసిన సుద్దాల అశోక్ తేజ గారికి \nపాట కి ప్రాణం పోసిన NTR గారికి నా పాదాభివందనం',
 'All are appreciating hero and his performance, I feel the song writer has put life into the song .. it creates a great sense of patriotism and selfless sacrifice to motherland ❤️❤️🙏',
 'మెగాస్టార్ అభిమానిగా చెప్తున్నాను...NTR నటన అత్యద్భుతం...! నభూతో అన్న రీతిలో నటవిశ్వరూపం చూపించాడు..NTR మినహా మిగతా ఎవరిని కూడా ఊహిచుకొలేక పోతున్నాను...!తెలుగు కళామతల్లిని అభిమానించే అందరిదీ ఈ విజయం👍',
 'నిముంద

In [208]:
len(comments)

1118

In [209]:
# to scrape time when comment was posted
tm_tags = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500]    

['',
 '1.5K',
 '',
 '1.1K',
 '',
 '525',
 '',
 '169',
 '',
 '426',
 '',
 '16',
 '',
 '925',
 '',
 '681',
 '',
 '249',
 '',
 '2',
 '',
 '498',
 '',
 '92',
 '',
 '2.6K',
 '',
 '402',
 '',
 '305',
 '',
 '33',
 '',
 '67',
 '',
 '585',
 '',
 '82',
 '',
 '6',
 '',
 '8.6K',
 '',
 '156',
 '',
 '57',
 '',
 '152',
 '',
 '5',
 '',
 '70',
 '',
 '3.8K',
 '',
 '237',
 '',
 '52',
 '',
 '36',
 '',
 '122',
 '',
 '72',
 '',
 '575',
 '',
 '70',
 '',
 '97',
 '',
 '29',
 '',
 '36',
 '',
 '37',
 '',
 '528',
 '',
 '90',
 '',
 '8.3K',
 '',
 '78',
 '',
 '71',
 '',
 '46',
 '',
 '61',
 '',
 '51',
 '',
 '289',
 '',
 '38',
 '',
 '121',
 '',
 '2',
 '',
 '43',
 '',
 '3',
 '',
 '488',
 '',
 '86',
 '',
 '4',
 '',
 '3',
 '',
 '22',
 '',
 '39',
 '',
 '2K',
 '',
 '65',
 '',
 '903',
 '',
 '40',
 '',
 '5',
 '',
 '3',
 '',
 '17',
 '',
 '2',
 '',
 '896',
 '',
 '21',
 '',
 '7',
 '',
 '14',
 '',
 '41',
 '',
 '29',
 '',
 '849',
 '',
 '27',
 '',
 '21',
 '',
 '5',
 '',
 '27',
 '',
 '21',
 '',
 '277',
 '',
 '19',
 '',
 '864',
 '',

In [210]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = comment_time[:500]
Youtube['Comment upvotes'] = No_of_Likes[:500]

#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,ఆడియో సాంగ్ రిలీజ్ అయినప్పుడు ఏం రెస్పాన్స్ రా...,2 weeks ago,1.5K
1,ఆయుధాలే కాదు..... పాట కూడా ప్రజలతో యుద్ధం చేయి...,11 days ago (edited),1.1K
2,Its NTR revolution ప్రపంచ సిని చరిత్రలో నటనె...,12 days ago,525
3,NTR fans అయినందుకు చాలా గర్వంగా ఉంది\nజై NTR A...,2 weeks ago,169
4,ఈ పాట విన్నంత సేపు కళ్ళలో నీళ్ళు ఆగలేదు 😭😭😭\nఈ...,2 weeks ago,426
...,...,...,...
495,NtR is IOVE U❤️❤️🤩🤩😍😍❤️😘✊✊👌👌👍🏼,2 weeks ago,2
496,Wonderful song,2 weeks ago,
497,Jai NTR 🔥🔥🔥,2 weeks ago,
498,Take a bow to NTR's performance and Charan's e...,2 weeks ago,56


Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.


In [211]:
# to get web browser
driver=webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver_win32_new\chromedriver.exe") 
time.sleep(3)

# to get url
url = "https://www.hostelworld.com/"
driver.get(url)

In [214]:
#to scrape location search bar
search_loc = driver.find_element_by_class_name('search-input')
search_loc.send_keys("London")
time.sleep(2)

In [215]:
#select london
london = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(2)

In [219]:
# do click in search button
search_btn = driver.find_element_by_id("search-button")
search_btn.click()

In [220]:
#empty lists
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []

In [223]:

for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    #to get hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #to get distance from city centre
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #to get privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #to get  dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #to get facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #to fetch url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #to click on show more button for description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #to get ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    # to get total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #to get  overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #to get property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [225]:
#to make dataframe
dff = pd.DataFrame({})
dff['Hostel_Name'] = hostel_name
dff['Distance fron city centre'] = distance
dff['Ratings'] = rating
dff['Total_reviews'] = reviews
dff['Overall Reviews'] = over_all
dff['Privates from price'] = pvt_prices
dff['Dorms from price'] = dorms_price
dff['Facilities'] = facilities[:58]
dff['Description'] = description

In [226]:
#Printing data frame
dff

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
0,Palmers Lodge - Swiss Cottage,6.5km from city centre,9.0,15338,Superb,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","40 College Crescent, Swiss Cottage, London, En..."
1,St Christopher's Village,1.8km from city centre,8.1,11190,Fabulous,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","165 Borough High Street, London, England"
2,Selina Camden,5.5km from city centre,9.3,24,Superb,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","89 Chalk Farm Rd, Chalk Farm, London, England"
3,Generator London,3km from city centre,7.7,6933,Very Good,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","Compton Place, Off 37 Tavistock Place, WC1, Lo..."
4,Astor Museum Inn,2.2km from city centre,8.6,8939,Fabulous,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","27 Montague Street, Bloomsbury, London, England"
5,Urbany Hostel London,5.4km from city centre,9.4,304,Superb,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","48-49 Princes Square, Paddington, London, England"
6,Astor Hyde Park,4.3km from city centre,8.5,11482,Fabulous,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","191 Queensgate, South Kensington, London, England"
7,Safestay London Elephant & Castle,1.7km from city centre,7.4,4233,Very Good,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","144-152 Walworth Road, London, England"
8,No.8 Willesden Hostel London,10km from city centre,7.0,4740,Very Good,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","305 High Rd, Willesden, London, England"
9,Smart Russell Square Hostel,2.6km from city centre,6.8,9640,Good,Rs20563.26 Rs17479,Rs2002,"Free WiFi, Follows Covid-19 sanitation guidance","71 Guilford Street, London, England"


In [227]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="3a9e60a81c973d308873821d76929be2")>>